# Web Scraping: Daily Mail

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import time

### Obtain list of news from the coverpage

URL definition:

In [2]:
# url definition
url = "https://www.investopedia.com/articles/investing/052014/how-googles-selfdriving-car-will-change-everything.asp"

List of news:

In [3]:
# Request
r1 = requests.get(url)
r1.status_code

# We'll save in coverpage the cover page content
newspage = r1.content

# Soup creation
soup1 = BeautifulSoup(newspage, 'html5lib')

# News identification
news_titles = soup1.find_all('span', class_ = "mntl-sc-block-heading__text")
news_titles = news_titles[3:6]

In [4]:
news_titles1 = [news_title.get_text() for news_title in news_titles]
news_titles1

[' Infrastructure Transformation ',
 ' Changing Oil Demand ',
 ' Safety Dividend ']

In [5]:
soup_article = BeautifulSoup(newspage, 'html5lib')
body = soup_article.find_all('p', class_ = "comp mntl-sc-block finance-sc-block-html mntl-sc-block-html")
temp = "".join(map (lambda x: x.get_text() ,body[17:26]))
temp

"\n“These vehicles should practice very efficient eco-driving practices, which is typically about 20% better than the average driver,” said Chase. “On the other hand, if these cars are owned by individuals, I see a huge rise in the number of trips, and vehicle miles traveled. People will send out their car to run errands they would never do if they had to be in the car and waste their own time. If the autonomous cars are shared vehicles and people pay for each trip, I think this will reduce demand, and thus (vehicle miles traveled).”\n\nAutonomous vehicles are also expected to be safer. “These cars won't get drunk or high, drive too fast, or take unnecessary risks—things people do all the time,” Chase said.\n\n“Over 90% of accidents today are caused by driver error,” said Professor Robert W. Peterson of the Center for Insurance Law and Regulation at Santa Clara University School of Law. “There is every reason to believe that self-driving cars will reduce the frequency and severity of a

In [6]:
bodies = [x.get_text() for x in body ]
bodies

['\nImagine getting into your car, typing—or, better yet speaking—a location into your vehicle’s interface, then letting it drive you to your destination while you read a book, surf the web, or nap. Self-driving vehicles—the stuff of science fiction since the first roads were paved—are coming, and they’re going to radically change what it’s like to get from point A to point B.\n',
 "\nIn 2009, Google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes. In 2016, Waymo, an autonomous driving technology company, became a subsidiary of Alphabet, and Google's self-driving project became Waymo.\n",
 '\nSince then, Waymo has invited the public to join the first public trial of autonomous vehicles operated by the Waymo Driver and introduced its first fully autonomous vehicles operated by the Waymo Driver on public roads without anyone in the driver’s seat.\n',
 '\n“The building blocks of driverless cars are on the road now,” explain

In [7]:
def file_writer(num,title,body):
    filename = '00'+str(num+1)+".txt"
    file = open(filename,"w")
    n = file.write(title + "\n\n"+body )
    file.close
for i in range(0,3):
    file_writer(i,news_titles1[i], bodies[i])

Now we have a list in which every element is a news article:

In [6]:
coverpage_news[0]

<h2 class="entry-title qodef-post-title" itemprop="name">
    
	        Five challenges in designing a fully autonomous system for driverless cars        	
</h2>

### Let's extract the text from the articles:

First, we'll define the number of articles we want:

In [7]:
number_of_articles = 1

In [11]:
title = coverpage_news[0].get_text()
title

'\n    \n\t        Five challenges in designing a fully autonomous system for driverless cars        \t\n'

In [14]:
soup_article = BeautifulSoup(coverpage, 'html5lib')
body = soup_article.find_all('p')
body[6]

<p>Autonomous cars would have to get onto the road where they would have to drive in all sorts of traffic conditions. They would have to drive with other autonomous cars on the road, and at the same time, there would also be a lot of humans. Wherever humans are involved, there are involved a lot of emotions. Traffic could be highly moderated and self-regulated. But often there are cases where people may be breaking traffic rules. An object may turn up in unexpected conditions. In the case of dense traffic, even the movement of few cms per minute does matter. One can’t wait endlessly for traffic to automatically clear and have some precondition to start moving. If more of such cars on the road are waiting for traffic to get cleared, ultimately that may result in a traffic deadlock.</p>

In [8]:
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

for n in np.arange(0, number_of_articles):
        
    # Getting the link of the article
    link = url + coverpage_news[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('p', class_='mol-para-with-font')
    
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(body)):
        paragraph = body[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    # Removing special characters
    final_article = re.sub("\\xa0", "", final_article)
        
    news_contents.append(final_article)

TypeError: 'NoneType' object is not subscriptable

Let's put them into:
* a dataset which will the input of the models (`df_features`)
* a dataset with the title and the link (`df_show_info`)

In [7]:
# df_features
df_features = pd.DataFrame(
     {'Article Content': news_contents 
    })

# df_show_info
df_show_info = pd.DataFrame(
    {'Article Title': list_titles,
     'Article Link': list_links})

In [8]:
df_features

,Article Content
0,A pair of student drug dealers have been spare...
1,Abu Hamza's son is the suspect held in connect...
2,The doorman stabbed in Mayfair on New Year's E...
3,Quite a lot of space on the internet has been ...
4,Grace Kelly's granddaughterCharlotte Casiraghi...


In [9]:
df_show_info

,Article Title,Article Link
0,Student drug dealers are SPARED jail after imp...,https://www.dailymail.co.uk/news/article-65535...
1,"Revealed: Hate preacher Abu Hamza's son, 26, i...",https://www.dailymail.co.uk/news/article-65540...
2,Doorman stabbed in Mayfair on New Year's Eve d...,https://www.dailymail.co.uk/news/article-65485...
3,"Enjoy crispy hash browns, the freshest fries, ...",https://www.dailymail.co.uk/femail/food/articl...
4,Grace Kelly's granddaughter Charlotte Casiragh...,https://www.dailymail.co.uk/femail/article-655...


### Time Elapsed

We are interested in how much time the script takes to get the news because this will impact directly on user experience. For this, we'll put it all into a single function and then call it:

In [10]:
def get_news_dailymail():
    
    # url definition
    url = "https://www.dailymail.co.uk"
    
    # Request
    r1 = requests.get(url)
    r1.status_code

    # We'll save in coverpage the cover page content
    coverpage = r1.content

    # Soup creation
    soup1 = BeautifulSoup(coverpage, 'html5lib')

    # News identification
    coverpage_news = soup1.find_all('h2', class_='linkro-darkred')
    len(coverpage_news)
    
    number_of_articles = 5

    # Empty lists for content, links and titles
    news_contents = []
    list_links = []
    list_titles = []

    for n in np.arange(0, number_of_articles):

        # Getting the link of the article
        link = url + coverpage_news[n].find('a')['href']
        list_links.append(link)

        # Getting the title
        title = coverpage_news[n].find('a').get_text()
        list_titles.append(title)

        # Reading the content (it is divided in paragraphs)
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        body = soup_article.find_all('p', class_='mol-para-with-font')

        # Unifying the paragraphs
        list_paragraphs = []
        for p in np.arange(0, len(body)):
            paragraph = body[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

         # Removing special characters
        final_article = re.sub("\\xa0", "", final_article)
        
        news_contents.append(final_article)
        

    # df_features
    df_features = pd.DataFrame(
         {'Content': news_contents 
        })

    # df_show_info
    df_show_info = pd.DataFrame(
        {'Article Title': list_titles,
         'Article Link': list_links,
         'Newspaper': 'Daily Mail'})
    
    return (df_features, df_show_info)

In [13]:
start = time.time()
x, y = get_news_dailymail()
end =time.time()
te = end-start
print("The time elapsed is %f seconds" %(te))

The time elapsed is 31.879444 seconds


Really slow. We won't include in the app.